In [1]:
import dill
dill.load_session('lqh_session.db')

In [1]:
import pandas as pd 
import numpy as np

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error


In [132]:
path = 'C:/Users/isaac/Dropbox/Apps/ShareLaTeX/information_lawyer_quality'

# Read data 
data = pd.read_csv(path + '/_aux/betas_score_hd.csv') 
#data = data.sample(frac=0.1, replace=True, random_state=1)
print(data.shape)
data.columns

(9755, 91)


Index(['junta', 'exp', 'anio', 'id', 'gen', 'horas_sem', 'salario_diario',
       'abogado_pub', 'c_antiguedad', 'trabajador_base', 'indem', 'sal_caidos',
       'prima_antig', 'prima_vac', 'hextra', 'prima_dom', 'desc_sem',
       'desc_ob', 'sarimssinf', 'utilidades', 'nulidad', 'codem', 'reinst',
       'salario_base_diario', 'c_indem', 'c_prima_antig', 'c_rec20', 'c_ag',
       'c_vac', 'c_hextra', 'c_prima_vac', 'c_prima_dom', 'c_desc_sem',
       'c_desc_ob', 'c_utilidades', 'c_recsueldo', 'c_total', 'c_sal_caidos',
       'min_ley', 'giro_empresa', 'tipo_persona_demandado',
       'dummy_correccion_domicilio', 'dummy_ampliacion_modif', 'dummy_exhorto',
       'contador_notificacion_actor', 'comparecencia_cde_actor',
       'contador_cde', 'comparecencia_oap_actor', 'contador_oap',
       'comparecencia_dp_actor', 'contador_dp', 'alegatos_actor',
       'confes_ofr_actor', 'testi_ofr_actor', 'docum_ofr_actor',
       'inspec_ofr_actor', 'peric_ofr_actor', 'confes_adm_actor',
    

In [2]:
data.describe()

,junta,exp,anio,id,gen,horas_sem,salario_diario,abogado_pub,c_antiguedad,trabajador_base,...,beta_pos_rec,beta_duration,beta_win_minley,beta_settlement,se_npv,se_liq_total,se_pos_rec,se_duration,se_win_minley,se_settlement
count,9753.000000,9753.000000,9753.000000,5361.000000,8690.000000,8557.000000,8651.000000,8687.000000,8617.000000,8669.000000,...,5274.000000,5274.000000,5274.000000,5252.000000,5274.000000,5274.000000,5274.000000,5274.000000,5274.000000,5252.000000
mean,8.968728,839.280734,2020.279196,106.938258,0.472382,57.096469,562.518601,0.101071,4.296889,0.076825,...,0.053813,-0.039548,-0.301046,0.035898,0.318095,0.469824,0.081637,0.115220,0.102940,0.084222
std,4.198952,506.250869,409.737185,61.943545,0.499265,15.305416,899.717722,0.301439,5.185201,0.266330,...,0.381048,0.435577,0.585693,0.373060,0.247758,0.358989,0.046018,0.082357,0.064415,0.049080
min,2.000000,1.000000,2011.000000,1.000000,0.000000,4.000000,3.950397,0.000000,0.000000,0.000000,...,-1.227233,-1.275068,-2.043814,-1.259683,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.000000,435.000000,2011.000000,44.000000,0.000000,48.000000,205.559768,0.000000,0.876712,0.000000,...,-0.044709,-0.263522,-0.625652,-0.105573,0.197895,0.327611,0.064731,0.083917,0.066483,0.067445
50%,9.000000,830.000000,2011.000000,116.000000,0.000000,60.000000,320.541412,0.000000,2.310000,0.000000,...,0.000000,-0.122162,-0.289188,0.009145,0.292161,0.440825,0.090239,0.115882,0.106671,0.090202
75%,11.000000,1194.000000,2011.000000,169.000000,1.000000,66.000000,590.935770,0.000000,5.540000,0.000000,...,0.276066,0.000000,0.000000,0.273846,0.417078,0.620215,0.106288,0.149174,0.135611,0.112389
max,16.000000,2120.000000,20111.000000,193.000000,1.000000,140.000000,29216.609460,1.000000,25.510000,1.000000,...,1.003252,1.944086,4.765472,1.076335,1.786304,3.374975,0.283458,0.749491,0.448801,0.376285


In [134]:
Xvars = ["gen",
            "horas_sem",
            "salario_diario",
            "abogado_pub",
            "c_antiguedad",
            "trabajador_base",
            "indem",
            "sal_caidos",
            "prima_antig",
            "prima_vac",
            "hextra",
            "prima_dom",
            "desc_sem",
            "desc_ob",
            "sarimssinf",
            "utilidades",
            "nulidad",
            "codem",
            "reinst",
            "salario_base_diario",
            "c_indem",
            "c_prima_antig",
            "c_rec20",
            "c_ag",
            "c_vac",
            "c_hextra",
            "c_prima_vac",
            "c_prima_dom",
            "c_desc_sem",
            "c_desc_ob",
            "c_utilidades",
            "c_recsueldo",
            "c_total",
            "c_sal_caidos",
            "min_ley",
            "giro_empresa"]
Zvars = ["tipo_persona_demandado",
            "dummy_correccion_domicilio",
            "dummy_ampliacion_modif",
            "dummy_exhorto",
            "contador_notificacion_actor",
            "comparecencia_cde_actor",
            "contador_cde",
            "comparecencia_oap_actor",
            "contador_oap",
            "comparecencia_dp_actor",
            "contador_dp",
            "alegatos_actor",
            "confes_ofr_actor",
            "testi_ofr_actor",
            "docum_ofr_actor",
            "inspec_ofr_actor",
            "peric_ofr_actor",
            "confes_adm_actor",
            "testi_adm_actor",
            "docum_adm_actor",
            "inspec_adm_actor",
            "peric_adm_actor",
            "confes_desahg_actor",
            "testi_desahog_actor",
            "docum_desahog_actor",
            "inspec_desahog_actor",
            "peric_desahog_actor",
            "confes_desahog_desist_actor",
            "testi_desahog_desist_actor",
            "docum_objecion_actor",
            "dummy_inspec_compar_actor",
            "dummy_inspec_mostro_actor"]

X = data[Xvars]
Z = data[Zvars]
W = pd.concat([X,Z], axis=1)

In [136]:
i = 0
y1 = pd.DataFrame()
y2 = pd.DataFrame()

for vdeps in ["npv", "liq_total", "pos_rec", "duration", "win_minley", "settlement"] :
    y1[i] = data[vdeps]
    y1.rename(columns={i: vdeps}, inplace=True)
    beta_col = "beta_" + vdeps 
    y2[i] = data[beta_col]
    y2.rename(columns={i: beta_col}, inplace=True)    
    i = i+1
 
    

In [137]:
numeric_featuresX = ["horas_sem",
            "salario_diario", 
            "c_antiguedad", 
            "salario_base_diario",
            "c_indem",
            "c_prima_antig",
            "c_rec20",
            "c_ag",
            "c_vac",
            "c_hextra",
            "c_prima_vac",
            "c_prima_dom",
            "c_desc_sem",
            "c_desc_ob",
            "c_utilidades",
            "c_recsueldo",
            "c_total",
            "c_sal_caidos",
            "min_ley"]

numeric_featuresZ = ["tipo_persona_demandado",
            "dummy_correccion_domicilio",
            "dummy_ampliacion_modif",
            "dummy_exhorto",
            "contador_notificacion_actor",
            "comparecencia_cde_actor",
            "contador_cde",
            "comparecencia_oap_actor",
            "contador_oap",
            "comparecencia_dp_actor",
            "contador_dp",
            "alegatos_actor",
            "confes_ofr_actor",
            "testi_ofr_actor",
            "docum_ofr_actor",
            "inspec_ofr_actor",
            "peric_ofr_actor",
            "confes_adm_actor",
            "testi_adm_actor",
            "docum_adm_actor",
            "inspec_adm_actor",
            "peric_adm_actor",
            "confes_desahg_actor",
            "testi_desahog_actor",
            "docum_desahog_actor",
            "inspec_desahog_actor",
            "peric_desahog_actor",
            "confes_desahog_desist_actor",
            "testi_desahog_desist_actor"]

numeric_featuresW = numeric_featuresX + numeric_featuresZ

In [138]:
categorical_featuresX = list(set(Xvars) - set(numeric_featuresX))
categorical_featuresZ = list(set(Zvars) - set(numeric_featuresZ))
categorical_featuresW = categorical_featuresX + categorical_featuresZ

In [159]:

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('polynomial', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler())])


categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse = False)


preprocessorX = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_featuresX),
        ('cat', categorical_transformer, categorical_featuresX)])
preprocessorZ = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_featuresZ),
        ('cat', categorical_transformer, categorical_featuresZ)])
preprocessorW = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_featuresW),
        ('cat', categorical_transformer, categorical_featuresW)])

In [179]:
# Define dependent variable 
Xy = pd.concat([X, y2['beta_liq_total']], axis=1)
Xy = Xy.dropna()

In [180]:
F = Xy.drop('beta_liq_total', axis=1)
y = Xy['beta_liq_total']

In [181]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(F, y, test_size=0.2,
                                                    random_state=0)

In [182]:
# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
from sklearn.linear_model import LinearRegression

In [183]:
# Linear Regression
LRX = Pipeline(steps=[('preprocessor', preprocessorX),
                      ('model', LinearRegression())])
LRX.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('polynomial',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['horas_sem',
                                                   'salario_diario',
                                                   'c_antiguedad',
                                                   'salario_base_diario',
                                                   'c_indem', 'c_prima_antig',
                       

In [197]:
from sklearn.ensemble import GradientBoostingRegressor

In [198]:
# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = {'model__n_estimators'  : [50, 100, 500, 1000],
              'model__max_features'  : ['auto', 'sqrt', 'log2'],
              'model__max_depth'     : [None, 1, 3, 5, 10, 20],
              'model__subsample'     : [0.5, 1],
              'model__learning_rate' : [0.001, 0.01, 0.1]
             }


GB  = GradientBoostingRegressor(
                        random_state        = 123,
                        # Activación de la parada temprana
                        validation_fraction = 0.1,
                        n_iter_no_change    = 5,
                        tol                 = 0.0001
                    )
# Gradient Boosting
GBX = Pipeline(steps=[('preprocessor', preprocessorX),
                      ('model', GB)])

# Búsqueda por grid search con validación cruzada
# ==============================================================================
gridX = GridSearchCV(GBX,
        param_grid = param_grid,
        scoring    = 'neg_root_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 4,
        return_train_score = True
       )

gridX.fit(X = X_train, y = y_train)
modelo_final_GBX = gridX.best_estimator_

Fitting 15 folds for each of 432 candidates, totalling 6480 fits


In [199]:
from sklearn.ensemble import RandomForestRegressor

In [200]:
param_grid = {'model__n_estimators': [100, 200, 500],
              'model__max_features': ["auto", "sqrt", "log2"],
              'model__max_depth'   : [None, 3, 10, 20],
              'model__criterion'   : ["mse", "mae"]
             }

# Random Forests
RFX = Pipeline(steps=[('preprocessor', preprocessorX),
                      ('model', RandomForestRegressor(random_state=123))])


# Búsqueda por grid search con validación cruzada
# ==============================================================================
gridX = GridSearchCV(
        estimator  = RFX,
        param_grid = param_grid,
        scoring    = 'neg_root_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 4,
        return_train_score = True
       )


gridX.fit(X = X_train, y = y_train)
modelo_final_RFX = gridX.best_estimator_

Fitting 15 folds for each of 72 candidates, totalling 1080 fits


In [165]:
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.cross_decomposition import PLSRegression
#from sklearn.pipeline import make_pipeline

In [184]:
# PCR
pcrX = Pipeline(steps=[('preprocessor', preprocessorX),
                     ('components', TruncatedSVD(n_components=10)),
                      ('model', LinearRegression())])


pcrX.fit(X_train, y_train)



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('polynomial',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['horas_sem',
                                                   'salario_diario',
                                                   'c_antiguedad',
                                                   'salario_base_diario',
                                                   'c_indem', 'c_prima_antig',
                       

In [185]:
from sklearn.preprocessing import FunctionTransformer

plsX = Pipeline(steps=[('preprocessor', preprocessorX),
                    ('model', PLSRegression(n_components=10))])



In [186]:
plsX.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('polynomial',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['horas_sem',
                                                   'salario_diario',
                                                   'c_antiguedad',
                                                   'salario_base_diario',
                                                   'c_indem', 'c_prima_antig',
                       

In [187]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from scipy.stats import uniform

In [188]:
# Espacio de búsqueda de cada hiperparámetro
# ==============================================================================
param_distributions = {
    'modelo__alpha': uniform(0.001, 10) ,
    'modelo__l1_ratio' : uniform(0.019, 0.99)
}

# Se combinan los pasos de preprocesado y el modelo en un mismo pipeline
ENX = Pipeline([('preprocessing', preprocessorX),
                 ('modelo', ElasticNet(random_state=123))])


# Búsqueda por validación cruzada
# ==============================================================================
gridX = RandomizedSearchCV(
        estimator  = ENX,
        param_distributions = param_distributions,
        n_iter     = 10,
        scoring    = 'neg_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123),  
        verbose    = 4,
        random_state = 123,
        return_train_score = True
       )


gridX.fit(X = X_train, y = y_train)
modelo_final_ENX = gridX.best_estimator_

Fitting 15 folds for each of 10 candidates, totalling 150 fits


In [189]:
# Espacio de búsqueda de cada hiperparámetro
# ==============================================================================
param_distributions = {
    'modelo__alpha': uniform(0.001, 10)
}

# Se combinan los pasos de preprocesado y el modelo en un mismo pipeline
LASSOX = Pipeline([('preprocessing', preprocessorX),
                 ('modelo', Lasso(random_state=123))])



# Búsqueda por validación cruzada
# ==============================================================================
gridX = RandomizedSearchCV(
        estimator  = LASSOX,
        param_distributions = param_distributions,
        n_iter     = 10,
        scoring    = 'neg_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123),  
        verbose    = 4,
        random_state = 123,
        return_train_score = True
       )


gridX.fit(X = X_train, y = y_train)
modelo_final_LASSOX = gridX.best_estimator_

Fitting 15 folds for each of 10 candidates, totalling 150 fits


In [190]:
# Se combinan los pasos de preprocesado y el modelo en un mismo pipeline
RIDGEX = Pipeline([('preprocessing', preprocessorX),
                 ('modelo', Ridge())])




# Búsqueda por validación cruzada
# ==============================================================================
gridX = RandomizedSearchCV(
        estimator  = RIDGEX,
        param_distributions = param_distributions,
        n_iter     = 10,
        scoring    = 'neg_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123),  
        verbose    = 4,
        random_state = 123,
        return_train_score = True
       )

gridX.fit(X = X_train, y = y_train)
modelo_final_RIDGEX = gridX.best_estimator_


Fitting 15 folds for each of 10 candidates, totalling 150 fits


In [201]:
import dill
dill.dump_session('lqh_session.db')

In [3]:
#Metric Results

metric_results_oosX = pd.DataFrame({'Linear Regression' :  [explained_variance_score(y_test, LRX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, LRX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, LRX.predict(X_test)),
    mean_squared_error(y_test, LRX.predict(X_test)),
    mean_absolute_percentage_error(y_test, LRX.predict(X_test))], 
                                     'GB' :  [explained_variance_score(y_test, modelo_final_GBX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_GBX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_GBX.predict(X_test)),
    mean_squared_error(y_test, modelo_final_GBX.predict(X_test)),
    mean_absolute_percentage_error(y_test, modelo_final_GBX.predict(X_test))],
                                    'RF' :  [explained_variance_score(y_test, modelo_final_RFX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_RFX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_RFX.predict(X_test)),
    mean_squared_error(y_test, modelo_final_RFX.predict(X_test)),
    mean_absolute_percentage_error(y_test, modelo_final_RFX.predict(X_test))],
                                    'PC Regression' :  [explained_variance_score(y_test, pcrX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, pcrX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, pcrX.predict(X_test)),
    mean_squared_error(y_test, pcrX.predict(X_test)),
    mean_absolute_percentage_error(y_test, pcrX.predict(X_test))], 
                                    'PLS' :  [explained_variance_score(y_test, plsX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, plsX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, plsX.predict(X_test)),
    mean_squared_error(y_test, plsX.predict(X_test)),
    mean_absolute_percentage_error(y_test, plsX.predict(X_test))], 
                                     'ElasticNet' :  [explained_variance_score(y_test, modelo_final_ENX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_ENX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_ENX.predict(X_test)),
    mean_squared_error(y_test, modelo_final_ENX.predict(X_test)),
    mean_absolute_percentage_error(y_test, modelo_final_ENX.predict(X_test))],
                                     'LASSO' :  [explained_variance_score(y_test, modelo_final_LASSOX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_LASSOX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_LASSOX.predict(X_test)),
    mean_squared_error(y_test, modelo_final_LASSOX.predict(X_test)),
    mean_absolute_percentage_error(y_test, modelo_final_LASSOX.predict(X_test))],
                                     'RIDGE' :  [explained_variance_score(y_test, modelo_final_RIDGEX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_RIDGEX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_RIDGEX.predict(X_test)),
    mean_squared_error(y_test, modelo_final_RIDGEX.predict(X_test)),
    mean_absolute_percentage_error(y_test, modelo_final_RIDGEX.predict(X_test))]                                 
                        })    

In [4]:
metric_results_insX = pd.DataFrame({'Linear Regression' :  [explained_variance_score(y_train, LRX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, LRX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, LRX.predict(X_train)),
    mean_squared_error(y_train, LRX.predict(X_train)),
    mean_absolute_percentage_error(y_train, LRX.predict(X_train))], 
                                     'GB' :  [explained_variance_score(y_train, modelo_final_GBX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_GBX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_GBX.predict(X_train)),
    mean_squared_error(y_train, modelo_final_GBX.predict(X_train)),
    mean_absolute_percentage_error(y_train, modelo_final_GBX.predict(X_train))],
                                    'RF' :  [explained_variance_score(y_train, modelo_final_RFX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_RFX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_RFX.predict(X_train)),
    mean_squared_error(y_train, modelo_final_RFX.predict(X_train)),
    mean_absolute_percentage_error(y_train, modelo_final_RFX.predict(X_train))],
                                    'PC Regression' :  [explained_variance_score(y_train, pcrX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, pcrX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, pcrX.predict(X_train)),
    mean_squared_error(y_train, pcrX.predict(X_train)),
    mean_absolute_percentage_error(y_train, pcrX.predict(X_train))], 
                                    'PLS' :  [explained_variance_score(y_train, plsX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, plsX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, plsX.predict(X_train)),
    mean_squared_error(y_train, plsX.predict(X_train)),
    mean_absolute_percentage_error(y_train, plsX.predict(X_train))], 
                                     'ElasticNet' :  [explained_variance_score(y_train, modelo_final_ENX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_ENX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_ENX.predict(X_train)),
    mean_squared_error(y_train, modelo_final_ENX.predict(X_train)),
    mean_absolute_percentage_error(y_train, modelo_final_ENX.predict(X_train))],
                                     'LASSO' :  [explained_variance_score(y_train, modelo_final_LASSOX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_LASSOX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_LASSOX.predict(X_train)),
    mean_squared_error(y_train, modelo_final_LASSOX.predict(X_train)),
    mean_absolute_percentage_error(y_train, modelo_final_LASSOX.predict(X_train))],
                                     'RIDGE' :  [explained_variance_score(y_train, modelo_final_RIDGEX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_RIDGEX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_RIDGEX.predict(X_train)),
    mean_squared_error(y_train, modelo_final_RIDGEX.predict(X_train)),
    mean_absolute_percentage_error(y_train, modelo_final_RIDGEX.predict(X_train))]                                       
                        })     
 

In [5]:
# Change the row indexes
metric_results_oosX.index = ['Explained Variance', 'R2', 'MAE', 'MSE', 'MAPE']
metric_results_insX.index = ['Explained Variance', 'R2', 'MAE', 'MSE', 'MAPE']

In [157]:
metric_results_oosZ

,Linear Regression,PC Regression,PLS,ElasticNet,LASSO,RIDGE
Explained Variance,-9.957223e+20,8.153956e-02,1.928932e-01,3.316196e-02,1.110223e-16,2.356853e-01
R2,-9.968598e+20,8.009401e-02,1.911608e-01,3.134862e-02,-1.859505e-03,2.343744e-01
MAE,1.081961e+10,2.003624e+00,1.839943e+00,2.059364e+00,2.096709e+00,1.747516e+00
MSE,1.121164e+22,1.034614e+01,9.096978e+00,1.089438e+01,1.126787e+01,8.610957e+00
MAPE,1.112350e+25,2.151258e+15,2.054936e+15,2.280374e+15,2.318560e+15,1.942750e+15


In [158]:
metric_results_insZ

,Linear Regression,PC Regression,PLS,ElasticNet,LASSO,RIDGE
Explained Variance,2.337493e-01,5.717490e-02,1.551787e-01,2.798975e-02,0.000000e+00,2.128211e-01
R2,2.337363e-01,5.717490e-02,1.551787e-01,2.798975e-02,0.000000e+00,2.128211e-01
MAE,1.669677e+00,1.964979e+00,1.823950e+00,2.006917e+00,2.042022e+00,1.705432e+00
MSE,6.607170e+00,8.129585e+00,7.284539e+00,8.381236e+00,8.622580e+00,6.787512e+00
MAPE,2.024017e+15,2.382052e+15,2.238907e+15,2.470778e+15,2.505048e+15,2.098903e+15


In [177]:
metric_results_oosW

,Linear Regression,PC Regression,PLS,ElasticNet,LASSO,RIDGE
Explained Variance,-6.217188e+14,5.555747e-02,5.407075e-01,3.624867e-02,0.000000e+00,7.307796e-01
R2,-6.247192e+14,4.574412e-02,5.405743e-01,2.295349e-02,-3.570600e-02,7.300085e-01
MAE,8.184790e+06,1.854334e+00,9.183162e-01,1.852341e+00,1.999300e+00,6.195618e-01
MSE,3.957359e+15,6.044849e+00,2.910288e+00,6.189219e+00,6.560805e+00,1.710294e+00
MAPE,1.086099e+21,2.686443e+15,6.704608e+14,2.632065e+15,2.569007e+15,5.448780e+14


In [8]:
metric_results_insW

NameError: name 'metric_results_insW' is not defined

In [6]:
metric_results_oosX

,Linear Regression,GB,RF,PC Regression,PLS,ElasticNet,LASSO,RIDGE
Explained Variance,-1.222930e+05,8.123019e-01,8.359444e-01,2.741809e-01,6.190101e-01,-2.220446e-16,-2.220446e-16,6.788465e-01
R2,-1.225411e+05,8.122494e-01,8.359421e-01,2.736874e-01,6.178702e-01,-2.645502e-03,-2.645502e-03,6.775185e-01
MAE,5.594178e+01,6.898068e-01,5.445988e-01,2.036226e+00,1.039326e+00,2.126502e+00,2.126502e+00,9.513450e-01
MSE,1.436751e+06,2.201292e+00,1.923507e+00,8.515694e+00,4.480303e+00,1.175558e+01,1.175558e+01,3.780953e+00
MAPE,4.849502e+14,5.915593e+14,3.350382e+14,2.712400e+15,5.192939e+14,2.624707e+15,2.624707e+15,4.746786e+14


In [7]:
metric_results_insX

,Linear Regression,GB,RF,PC Regression,PLS,ElasticNet,LASSO,RIDGE
Explained Variance,7.912220e-01,9.526423e-01,9.684005e-01,1.624619e-01,6.371104e-01,0.000000e+00,0.000000e+00,7.054536e-01
R2,7.912220e-01,9.526239e-01,9.683855e-01,1.624619e-01,6.371104e-01,0.000000e+00,0.000000e+00,7.054536e-01
MAE,7.007522e-01,3.191382e-01,2.162753e-01,2.039077e+00,9.610832e-01,2.086022e+00,2.086022e+00,8.455850e-01
MSE,1.800400e+00,4.085481e-01,2.726281e-01,7.222521e+00,3.129383e+00,8.623513e+00,8.623513e+00,2.540025e+00
MAPE,4.913612e+14,3.777658e+14,1.219234e+14,2.887727e+15,5.676155e+14,2.820578e+15,2.820578e+15,4.982756e+14
